# Implementation of the Fast Gradient Sign Method (FGSM)

## 1. Loading the model and images to attack
We will use the model `VGG16`, trained on the `ImageNet` dataset, to conduct our experiments.

## 2. Implementation of FGSM for confidence reduction
We implement the simplest case of FGSM, using the following equation:
$$X_* = X + \varepsilon\text{ sign}\left(\nabla_x J(X, y_{\text{true}})\right)$$

## 3. Variation of FGSM for source/target misclassification
We modify FGSM to specifically choose the target class, i.e. the class that we want our crafted image to be classified as:
$$X_* = X - \varepsilon\text{ sign}\left(\nabla_x J(X, y_{\text{target}})\right)$$

## 4. Iterating FGSM
Finally, we iterate FGSM over multiple steps of gradient evaluation, and compare the results to the single-step setup:
$$\begin{cases}
        X_*^0 \quad= X\\
        X^{n+1}_* = \text{Clip}\left(X^n_*+\alpha\text{ sign}\left(\nabla_x J(X, y_{\text{true}})\right)\right)
\end{cases}$$